In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

### Fetch the results of RPGF round 4, 5, and 6

In [4]:
def fetch_optimism_round_data(round_number):
    url = f"https://atlas.optimism.io/round/results?rounds={round_number}"
    print(f"Fetching data for Round {round_number}...")

    # Setup headless browser
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)

    driver.get(url)

    # Click "Show more" until all content is loaded
    while True:
        try:
            show_more = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Show more')]")))
            driver.execute_script("arguments[0].click();", show_more)
            time.sleep(2)
        except:
            print("All projects loaded.")
            break

    # Get the full page HTML
    html = driver.page_source
    driver.quit()

    # Parse with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    data = []

    # Locate all project cards
    project_blocks = soup.find_all('div', class_='flex flex-row justify-between py-8 gap-1')

    for block in project_blocks:
        a_tag = block.find_parent('a', class_='block transition-colors group')
        link = "https://atlas.optimism.io" + a_tag['href'] if a_tag and a_tag.has_attr('href') else "N/A"

        h5 = block.find('h5')
        project_name = h5.get_text(strip=True) if h5 else "N/A"

        p = block.find('p')
        about_project = p.get_text(separator="\n", strip=True) if p else "N/A"

        spans = block.find_all('span')
        fund_value = "N/A"
        for span in spans[::-1]:
            text = span.get_text(strip=True).replace(',', '')
            if text.replace('.', '', 1).isdigit():
                fund_value = text
                break

        data.append({
            "project_name": project_name,
            "project_description": about_project,
            "funds_allocated": fund_value,
            "project_profile": link
        })

    # Save to CSV
    df = pd.DataFrame(data)
    filename = f"../Data/rpgf{round_number}_results.csv"
    df.to_csv(filename, index=False)
    print(f"Round {round_number}: Saved {len(df)} projects to {filename}\n")

# --- Run for Rounds 4, 5 and 6 ---
fetch_optimism_round_data(4)
fetch_optimism_round_data(5)
fetch_optimism_round_data(6)


Fetching data for Round 4...
All projects loaded.
Round 4: Saved 207 projects to ../Data/rpgf4_results.csv

Fetching data for Round 5...
All projects loaded.
Round 5: Saved 79 projects to ../Data/rpgf5_results.csv

Fetching data for Round 6...
All projects loaded.
Round 6: Saved 88 projects to ../Data/rpgf6_results.csv



### Format the data of RPGF round 1, 2, and 3

In [5]:
round_1 = pd.read_csv("../Data/rpgf1_results.csv")
round_2 = pd.read_csv("../Data/rpgf2_results.csv")
round_3 = pd.read_csv("../Data/rpgf3_results.csv")
round_4 = pd.read_csv("../Data/rpgf4_results.csv")
round_5 = pd.read_csv("../Data/rpgf5_results.csv")
round_6 = pd.read_csv("../Data/rpgf6_results.csv")

In [6]:
print("RPGF 1:", round_1.columns)
print("\nRPGF 2:", round_2.columns)
print("\nRPGF 3:", round_3.columns)
print("\nRPGF 4:", round_4.columns)
print("\nRPGF 5:", round_5.columns)
print("\nRPGF 6:", round_6.columns)

RPGF 1: Index(['Project', '$ Value Awarded'], dtype='object')

RPGF 2: Index(['Project Name', 'Category', '% of votes received', 'OP Received'], dtype='object')

RPGF 3: Index(['Project Name', 'ballots', 'median amount', 'Quorum reached',
       'OP Received', 'Project Profile'],
      dtype='object')

RPGF 4: Index(['project_name', 'project_description', 'funds_allocated',
       'project_profile'],
      dtype='object')

RPGF 5: Index(['project_name', 'project_description', 'funds_allocated',
       'project_profile'],
      dtype='object')

RPGF 6: Index(['project_name', 'project_description', 'funds_allocated',
       'project_profile'],
      dtype='object')


In [7]:
# rename the columns
round_1 = round_1.rename(columns={
    'Project': 'project_name',
    '$ Value Awarded': 'funds_allocated'
})

round_2 = round_2.rename(columns={
    'Project Name': 'project_name',
    'OP Received': 'funds_allocated'
})

round_3 = round_3.rename(columns={
    'Project Name': 'project_name',
    'OP Received': 'funds_allocated',
    'Project Profile': 'project_profile'
})

In [8]:
# drop the unnecessary columns
round_2.drop(columns=['Category', '% of votes received'], inplace=True)

round_3.drop(columns=['ballots', 'median amount', 'Quorum reached'], inplace=True)

In [9]:
print("RPGF 1:", round_1.columns)
print("\nRPGF 2:", round_2.columns)
print("\nRPGF 3:", round_3.columns)

RPGF 1: Index(['project_name', 'funds_allocated'], dtype='object')

RPGF 2: Index(['project_name', 'funds_allocated'], dtype='object')

RPGF 3: Index(['project_name', 'funds_allocated', 'project_profile'], dtype='object')


In [10]:
# add the grant_type and round_number column
round_1['grant_type'] = 'Retro Funding'
round_2['grant_type'] = 'Retro Funding'
round_3['grant_type'] = 'Retro Funding'
round_4['grant_type'] = 'Retro Funding'
round_5['grant_type'] = 'Retro Funding'
round_6['grant_type'] = 'Retro Funding'

round_1['round_number'] = 1
round_2['round_number'] = 2
round_3['round_number'] = 3
round_4['round_number'] = 4
round_5['round_number'] = 5
round_6['round_number'] = 6

In [11]:
round_1.head()

,project_name,funds_allocated,grant_type,round_number
0,Ethersjs,"$51,345",Retro Funding,1
1,go-ethereum,"$45,232",Retro Funding,1
2,EthGlobal,"$42,787",Retro Funding,1
3,Hardhat,"$41,565",Retro Funding,1
4,WalletConnect,"$40,342",Retro Funding,1


In [12]:
round_2.head()

,project_name,funds_allocated,grant_type,round_number
0,Protocol Guild,"557,301.00",Retro Funding,2
1,L2BEAT,"256,294.36",Retro Funding,2
2,geth,"230,590.08",Retro Funding,2
3,ETHGlobal,"230,005.52",Retro Funding,2
4,BuidlGuidl,"224,174.11",Retro Funding,2


In [13]:
round_3.head()

,project_name,funds_allocated,project_profile,grant_type,round_number
0,Protocol Guild,663853.62,https://vote.optimism.io/retropgf/3/applicatio...,Retro Funding,3
1,go-ethereum,496896.42,https://vote.optimism.io/retropgf/3/applicatio...,Retro Funding,3
2,Solidity,422361.96,https://vote.optimism.io/retropgf/3/applicatio...,Retro Funding,3
3,Erigon,339545.72,https://vote.optimism.io/retropgf/3/applicatio...,Retro Funding,3
4,Lighthouse,298137.85,https://vote.optimism.io/retropgf/3/applicatio...,Retro Funding,3


In [14]:
round_4.head()

,project_name,project_description,funds_allocated,project_profile,grant_type,round_number
0,Layer3,Layer3 is solving the attention problem in cry...,500000,https://atlas.optimism.io/project/0x91a4420e2f...,Retro Funding,4
1,Zora,Zora is a new kind of social network to expres...,500000,https://atlas.optimism.io/project/0x9102357674...,Retro Funding,4
2,LI.FI,"One API to swap, bridge, and zap across all ma...",481556,https://atlas.optimism.io/project/0x517eaa9c56...,Retro Funding,4
3,Stargate Finance,Stargate is a fully composable liquidity\ntran...,426708,https://atlas.optimism.io/project/0x62e37e96aa...,Retro Funding,4
4,ODOS,Odos leverages an intent optimization algorith...,359091,https://atlas.optimism.io/project/0xd5b4c54b12...,Retro Funding,4


In [15]:
round_5.head()

,project_name,project_description,funds_allocated,project_profile,grant_type,round_number
0,go-ethereum,The go-ethereum team develops the software whi...,234545,https://atlas.optimism.io/project/0x79c2ae8858...,Retro Funding,5
1,Protocol Guild,Protocol Guild is a funding collective for 181...,223384,https://atlas.optimism.io/project/0xc49d46c560...,Retro Funding,5
2,Solidity,"Solidity is an object-oriented, high-level lan...",204725,https://atlas.optimism.io/project/0xcc8d03e014...,Retro Funding,5
3,Conduit OP Stack Contributions,The rollup-native cloud platform. Conduit give...,173765,https://atlas.optimism.io/project/0xc879d4a2a3...,Retro Funding,5
4,Account Abstraction - ERC-4337,The AA team is working on standards for decent...,163048,https://atlas.optimism.io/project/0xb98778ca9f...,Retro Funding,5


In [16]:
round_6.head()

,project_name,project_description,funds_allocated,project_profile,grant_type,round_number
0,OSO Insights & Data Science,Open Source Observer is a public good that hel...,87995,https://atlas.optimism.io/project/0x77cc2b19e7...,Retro Funding,6
1,numbaNERDs,The numbaNERDs Program is a governance-focused...,77381,https://atlas.optimism.io/project/0x431eb34f7f...,Retro Funding,6
2,Ethereum Attestation Service (EAS),EAS is an infrastructure public good for makin...,65620,https://atlas.optimism.io/project/0xa88844cea1...,Retro Funding,6
3,growthepie 🥧📏 Ethereum and Superchain Analytics,growthepie.xyz is a public goods data platform...,60955,https://atlas.optimism.io/project/0xa38f3efb4f...,Retro Funding,6
4,Optimism Developer Advisory Board (S6),The Developer Advisory Board (DAB) brings a te...,58280,https://atlas.optimism.io/project/0x74f635c5e7...,Retro Funding,6


### Merge all the rounds data

In [17]:
# List of all dataframes
all_rounds_data = [round_1, round_2, round_3, round_4, round_5, round_6]

# Define the expected column schema
expected_columns = [
    'project_name',
    'project_description',
    'funds_allocated',
    'project_profile',
    'grant_type',
    'round_number'
]

# Clean and align each round's DataFrame
aligned_dfs = []
for df in all_rounds_data:
    # Add missing columns with NA
    for col in expected_columns:
        if col not in df.columns:
            df[col] = 'N/A'
    # Reorder columns
    df = df[expected_columns]
    aligned_dfs.append(df)

# Concatenate all rounds
merged_df = pd.concat(aligned_dfs, ignore_index=True)

In [18]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1270 entries, 0 to 1269
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   project_name         1270 non-null   object
 1   project_description  1270 non-null   object
 2   funds_allocated      1270 non-null   object
 3   project_profile      1270 non-null   object
 4   grant_type           1270 non-null   object
 5   round_number         1270 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 59.7+ KB


In [19]:
#Remove unwanted characters (commas, $, etc.)
merged_df['funds_allocated'] = merged_df['funds_allocated'].replace(r'[^\d.]', '', regex=True)

# Convert to float
merged_df['funds_allocated'] = merged_df['funds_allocated'].astype(float)

# Format to 2 decimal places 
merged_df['funds_allocated'] = merged_df['funds_allocated'].round(2)

In [20]:
# Convert 'funds_allocated' to string
merged_df['funds_allocated'] = merged_df['funds_allocated'].astype(str)

# Apply formatting conditionally
merged_df['funds_allocated'] = merged_df.apply(
    lambda row: f"${row['funds_allocated']}" if row['round_number'] == 1 else f"{row['funds_allocated']} OP",
    axis=1
)

In [21]:
merged_df.head()

,project_name,project_description,funds_allocated,project_profile,grant_type,round_number
0,Ethersjs,N/A,$51345.0,N/A,Retro Funding,1
1,go-ethereum,N/A,$45232.0,N/A,Retro Funding,1
2,EthGlobal,N/A,$42787.0,N/A,Retro Funding,1
3,Hardhat,N/A,$41565.0,N/A,Retro Funding,1
4,WalletConnect,N/A,$40342.0,N/A,Retro Funding,1


In [22]:
merged_df.tail()

,project_name,project_description,funds_allocated,project_profile,grant_type,round_number
1265,Optimism Fractal Respect Game Events,Optimism Fractal is a community dedicated to f...,7649.0 OP,https://atlas.optimism.io/project/0x20617cf248...,Retro Funding,6
1266,Mission Request Workshop,"Hello I am Jesse (Jrocki), I currently serve o...",7649.0 OP,https://atlas.optimism.io/project/0x51fdeea1d3...,Retro Funding,6
1267,Syntra,Syntra is a delegate dashboard that simplifies...,7348.0 OP,https://atlas.optimism.io/project/0x09261dbfb4...,Retro Funding,6
1268,FrameMaker,FrameMaker - Tilda in the frames world of Warp...,4641.0 OP,https://atlas.optimism.io/project/0xf173b116c6...,Retro Funding,6
1269,Optimism Delegate Frame on Farcaster,The “Optimism Delegate Frame” on Farcaster is ...,4641.0 OP,https://atlas.optimism.io/project/0x3d7a2f24e6...,Retro Funding,6


In [23]:
merged_df.to_csv("../Data/RPGF_Results_Data.csv", index=False)

### Clean the text data and save it in a JSON file

In [24]:
# Clean text columns
text_columns = ['project_name', 'project_description', 'project_profile']

for col in text_columns:
    if col in merged_df.columns:
        # Convert to string, remove \n, and strip leading/trailing whitespace
        merged_df[col] = (
            merged_df[col]
            .astype(str)
            .str.replace('\n', ' ', regex=False)
            .str.replace('\r', ' ', regex=False) 
            .str.strip()
        )

# Convert to JSON string
json_str = merged_df.to_json(orient="records", indent=2, force_ascii=False)

# Remove escaped forward slashes for clean URLs
json_str = json_str.replace('\\/', '/')

# Save to a JSON file
output_path = "../Data/RPGF_Results_Data.json"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(json_str)

print(f"Clean JSON saved to: {output_path}")

Clean JSON saved to: ../Data/RPGF_Results_Data.json
